## Prediction Logic

In [14]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [15]:
with open('artifacts/Age_encoder.pkl','rb') as File:
    label_encoder = pickle.load(File)
    
with open('artifacts/onehot_encoder.pkl','rb') as File:
    onehot_encode = pickle.load(File) 
    
with open('artifacts/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [16]:
sample_input = {
    "Area": "India",
    "Item": "Rice, paddy",
    "Year": 2013,
    "average_rain_fall_mm_per_year": 1083.0,
    "pesticides_tonnes": 36000.0,
    "avg_temp": 25.5
}

In [17]:
input_df = pd.DataFrame([sample_input])
input_df

,Area,Item,Year,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,India,"Rice, paddy",2013,1083.0,36000.0,25.5


In [18]:
## Transform item
Item_encoder = onehot_encode.transform(input_df[['Item']]).toarray()
onehot_encode.get_feature_names_out(['Item'])
Item_encoded_df=pd.DataFrame(Item_encoder,columns=onehot_encode.get_feature_names_out(['Item']))
Item_encoded_df

,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
input_df=pd.concat([input_df.drop('Item',axis=1),Item_encoded_df],axis=1)
input_df

,Area,Year,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,India,2013,1083.0,36000.0,25.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
## Transform Area
input_df["Area_te"] = label_encoder.transform(input_df["Area"])
input_df.drop(columns=["Area"], inplace=True)
input_df


,Year,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams,Area_te
0,2013,1083.0,36000.0,25.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,81322.787458


In [22]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.61997505, -0.08628166, -0.01903089,  0.78755649, -0.27940044,
        -0.4137978 , -0.14125846, -0.42147216,  2.69232815, -0.34667628,
        -0.35676702, -0.33657795, -0.39886202, -0.17748953,  0.13530864]])

In [24]:
##Load the model
model = load_model('model.h5')

In [25]:
## Final Prediction
predict = model.predict(input_scaled)
predict

1/1 [==============================] - 0s 107ms/step


array([[35791.902]], dtype=float32)

In [26]:
prediction = predict[0][0]
print(prediction)

35791.902
